In [116]:
using System;
using XPlot.Plotly;

class AdExNeuron {
    // Following section 2.4.4 of Jolivet et al. (2008), J Neurosci methods
    public double C = 72E-12; // F, membrane time constant
    public double d_T = 2E-3; // V, threshold sharpness. In paper it's 1000 times smaller, but pretty sure it's a typo.
    public double V_T = -38E-3; // V, threshold voltage
    public double tau_w = 25E-3; // s, relaxation time of adaptation
    public double a = .5E-9; // S, conductance of adaptation variable
    public double b = 36E-12; // A, equivalent current added to adaptation variable after spike
    public double E_L = -60E-3; // V, reversal potential
    public double g_L = 13E-9; // S, leak conductance
    
    // Not actually written in paper, but in Table 1 of Naud et a. (2008), E_R ~= E_L + 10 mV
    public double E_R = -50E-3; // V, reset potential.
    
    public double threshold = 0.0; // V, threshold.
    
    public double u = 0.0; // V, voltage.
    public double w = 0.0; // A, adaptation current.
    
    public double timestep = 0.0001; // s, simulation timestep.
    
    public AdExNeuron() {
        // Start at the reversal potential.
        reset();
    }
    
    public bool SimulateOneStep(double I) {
        // Use a simple first order Euler method to move time forward.
        var dwdt = 1 / tau_w * (a * (u - E_L) - w);
        var dudt = 1 / C * (-g_L * (u - E_L) + g_L * d_T * Math.Exp((u - V_T) / d_T) + I - w);

        w = w + timestep * dwdt;
        u = u + timestep * dudt;
        
        bool spike = false;
        if(u > threshold) {
            u = E_R;
            w = w + b;
            spike = true;
        }
        return spike;
    }
    
    public void reset() {
        u = E_L;
    }
}

In [116]:
var neuron = new AdExNeuron();

var time = new List<double>();
var spike_times = new List<double>();
var us = new List<double>();

// Simulate a current step of 10 pA.
for(var i = 0; i < 10000; i++) {
    var I = 300E-12;
    if(i < 2500 || i > 7500) {
        I = 0;
    }
    var spike = neuron.SimulateOneStep(I);
    time.Add(i * neuron.timestep);
    if(spike) {
        spike_times.Add(time[time.Count - 1]);
    }
    us.Add(neuron.u);
}

var zeros = new List<double>();
foreach(var s in spike_times) {
    zeros.Add(0);
}

var subgraphs = new List<Graph.Scattergl>();
subgraphs.Add(new Graph.Scattergl()
    {
        x = time,
        y = us,
        mode = "lines",
        name = "Voltage trace"
    });
subgraphs.Add(new Graph.Scattergl()
    {
        x = spike_times,
        y = zeros,
        mode = "markers",
        name = "Spike times"
    });

var chart = Chart.Plot(
    subgraphs,
    new Layout.Layout()
    {
        title = "AdEx simulation with current step",
        xaxis = new Graph.Xaxis() {
            title = "time (s)"
        },
        yaxis = new Graph.Yaxis() {
            title = "voltage (V)"
        },
        legend = new Graph.Legend() {
            
        }
    }
);

chart.Width = 1200;
chart.Height = 600;
display(chart);